In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from torch.amp import GradScaler, autocast
from PIL import Image

def main():
    # 데이터 경로 설정 (최상위 폴더)
    data_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\data\category_data"
    
    # 모델 저장 경로 (상대 경로)
    model_save_path = "./models/category_model3.pth"

    # 하이퍼파라미터
    BATCH_SIZE = 128
    IMG_SIZE = (224, 224)
    EPOCHS = 30
    LEARNING_RATE = 0.001
    NUM_CLASSES = 10  # 클래스 개수 (자동 탐색)

    # 데이터 전처리
    transform = transforms.Compose([
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # ✅ 데이터셋 로드 (하위 폴더 내 이미지 자동 탐색)
    dataset = CustomCategoryDataset(data_dir, transform=transform)
    NUM_CLASSES = len(dataset.classes)  # 클래스 개수 자동 설정

    # ✅ 데이터 분할 (80% 훈련, 20% 검증)
    train_size = int(0.8 * len(dataset))
    valid_size = len(dataset) - train_size
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

    # ✅ EfficientNet-B0 모델 정의
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # ✅ 손실 함수 및 옵티마이저
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scaler = GradScaler("cuda")

    # ✅ 검증 정확도가 가장 높았던 모델 저장
    best_valid_accuracy = 0.0

    # ✅ 학습 루프
    for epoch in range(EPOCHS):
        print(f"\n=== Starting Epoch {epoch + 1}/{EPOCHS} ===")
        model.train()
        train_loss, train_correct = 0, 0

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item() * inputs.size(0)
            train_correct += (outputs.argmax(1) == labels).sum().item()

            progress = int((batch_idx + 1) / len(train_loader) * 100)
            print(f"Epoch {epoch + 1}/{EPOCHS}: {progress}% completed", end="\r")

        train_loss /= len(train_loader.dataset)
        train_accuracy = train_correct / len(train_loader.dataset)

        # ✅ Validation 루프
        valid_loss, valid_correct = 0, 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                valid_loss += loss.item() * inputs.size(0)
                valid_correct += (outputs.argmax(1) == labels).sum().item()

        valid_loss /= len(valid_loader.dataset)
        valid_accuracy = valid_correct / len(valid_loader.dataset)

        print(f"\nEpoch {epoch + 1}/{EPOCHS} completed.")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
            f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_accuracy:.4f}")

        # ✅ 모델 저장 (최고 정확도 업데이트 시)
        if valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = valid_accuracy
            save_model(model, optimizer, epoch + 1, train_loss, valid_loss, train_accuracy, valid_accuracy, model_save_path)

    print("\nTraining completed. Best model saved to:", model_save_path)

# ✅ CustomCategoryDataset에서 데이터 로드 상태 출력 추가
class CustomCategoryDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # ✅ 클래스 목록 가져오기 (최상위 폴더만)
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

        print(f"✅ 총 {len(self.classes)}개의 클래스 발견: {self.classes}")  # 디버깅 출력

        # ✅ 하위 폴더 내 이미지 자동 탐색
        for cls_name in self.classes:
            class_dir = os.path.join(root_dir, cls_name)
            found_images = 0
            for root, _, files in os.walk(class_dir):
                for file in files:
                    if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif', 'tiff', 'webp')):
                        self.image_paths.append(os.path.join(root, file))
                        self.labels.append(self.class_to_idx[cls_name])
                        found_images += 1

            if found_images == 0:
                print(f"⚠️ {cls_name} 폴더 내 이미지가 없음 (데이터셋 문제 가능)")

        print(f"✅ 총 {len(self.image_paths)}개의 이미지 로드 완료")  # 디버깅 출력

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # ✅ 이미지 파일이 정상적으로 열리는지 확인
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"🚨 오류: {img_path} 열 수 없음 ({e})")
            return None, None  # 데이터 로드 실패 시 None 반환

        if self.transform:
            image = self.transform(image)

        return image, label

def save_model(model, optimizer, epoch, train_loss, valid_loss, train_accuracy, valid_accuracy, model_save_path):
    """모델을 저장하는 함수"""
    model_dir = os.path.dirname(model_save_path)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        print(f"Created directory: {model_dir}")

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'valid_loss': valid_loss,
        'train_accuracy': train_accuracy,
        'valid_accuracy': valid_accuracy,
    }, model_save_path)
    print(f"Model saved to {model_save_path}")

if __name__ == "__main__":
    main()


✅ 총 10개의 클래스 발견: ['가방·박스·웨건', '공구·액세서리', '냉·난방용품', '랜턴', '매트·침낭', '식기·주방', '전기·전자기기', '테이블·체어', '텐트·타프', '화로대·버너']
✅ 총 13883개의 이미지 로드 완료

=== Starting Epoch 1/30 ===
Epoch 1/30: 100% completed
Epoch 1/30 completed.
Train Loss: 0.7227, Train Acc: 0.7688, Valid Loss: 0.5106, Valid Acc: 0.8408
Created directory: ./models
Model saved to ./models/category_model3.pth

=== Starting Epoch 2/30 ===
Epoch 2/30: 100% completed
Epoch 2/30 completed.
Train Loss: 0.2600, Train Acc: 0.9158, Valid Loss: 0.4078, Valid Acc: 0.8725
Model saved to ./models/category_model3.pth

=== Starting Epoch 3/30 ===
Epoch 3/30: 100% completed
Epoch 3/30 completed.
Train Loss: 0.1537, Train Acc: 0.9504, Valid Loss: 0.4542, Valid Acc: 0.8740
Model saved to ./models/category_model3.pth

=== Starting Epoch 4/30 ===
Epoch 4/30: 100% completed
Epoch 4/30 completed.
Train Loss: 0.1312, Train Acc: 0.9591, Valid Loss: 0.5093, Valid Acc: 0.8596

=== Starting Epoch 5/30 ===
Epoch 5/30: 100% completed
Epoch 5/30 completed.
Train

KeyboardInterrupt: 

In [5]:
import os
import torch
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import DataLoader
from collections import Counter

# ✅ 모델 및 데이터 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ 사용 중인 GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# ✅ 모델 및 데이터셋 설정
data_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/data/category_data"
model_path = "C:/Users/user/OneDrive/Desktop/Resnet18-real/model/best_category_model.pth"
csv_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/csv"

# ✅ 데이터셋 로드
from torchvision import datasets, transforms, models
from torchvision.models import EfficientNet_B0_Weights

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(data_dir, transform=transform)
valid_size = int(0.2 * len(dataset))
train_size = len(dataset) - valid_size
_, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# ✅ 모델 불러오기
model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, len(dataset.classes))
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# ✅ 검증 데이터셋에서 자주 틀린 데이터 찾기
def find_misclassified_samples(loader):
    misclassified = []
    class_misclassification = Counter()

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = outputs.argmax(1)

            for i in range(len(labels)):
                image_idx = idx * loader.batch_size + i
                pred_label = dataset.classes[predicted[i].item()]
                true_label = dataset.classes[labels[i].item()]

                if pred_label != true_label:
                    misclassified.append((image_idx, pred_label, true_label))
                    class_misclassification[true_label] += 1

    print("\n✅ 검증 데이터셋에서 자주 틀린 클래스 (상위 10개)")
    for label, count in class_misclassification.most_common(10):
        print(f"Class {label}: {count}번 오답 발생")

    return misclassified

# ✅ 클래스별 정확도 확인
def compute_classwise_accuracy(loader):
    class_correct = Counter()
    class_total = Counter()

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = outputs.argmax(1)

            for i in range(len(labels)):
                true_label = labels[i].item()
                class_total[true_label] += 1
                if predicted[i] == true_label:
                    class_correct[true_label] += 1

    class_accuracy = {}
    print("\n✅ 검증 데이터셋 클래스별 정확도:")
    for label in sorted(class_total.keys()):
        accuracy = class_correct[label] / class_total[label] if class_total[label] > 0 else 0
        class_accuracy[dataset.classes[label]] = accuracy
        print(f"Class {dataset.classes[label]}: {accuracy:.2%} 정확도")

    return class_accuracy

# ✅ 확신도가 낮은 샘플 찾기
def find_low_confidence_samples(loader, threshold=0.3):
    low_confidence_samples = []

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            confidence, predicted = probabilities.max(1)

            for i in range(len(labels)):
                image_idx = idx * loader.batch_size + i
                if confidence[i].item() < threshold:  # 🔥 확신도가 낮은 샘플 저장
                    low_confidence_samples.append((image_idx, dataset.classes[predicted[i].item()], dataset.classes[labels[i].item()], confidence[i].item()))

    print("\n✅ 검증 데이터셋에서 확신도가 낮은 샘플 (상위 10개)")
    for i, (image_idx, pred_label, true_label, conf) in enumerate(low_confidence_samples[:10]):
        print(f"이미지 {image_idx}: 예측={pred_label}, 실제={true_label}, 확신도={conf:.2f}")

    return low_confidence_samples

# ✅ 실행
print("\n🔥 검증 데이터셋에서 문제있는 데이터 찾기 🔥")
misclassified_valid = find_misclassified_samples(valid_loader)
class_accuracy = compute_classwise_accuracy(valid_loader)
low_confidence_valid = find_low_confidence_samples(valid_loader)

# ✅ CSV로 저장 (문제있는 데이터 정리)
misclassified_df = pd.DataFrame(misclassified_valid, columns=["Image Index", "Predicted Label", "True Label"])
low_confidence_df = pd.DataFrame(low_confidence_valid, columns=["Image Index", "Predicted Label", "True Label", "Confidence"])
class_accuracy_df = pd.DataFrame(list(class_accuracy.items()), columns=["Class", "Accuracy"])

misclassified_csv = os.path.join(csv_dir, "misclassified_samples.csv")
low_confidence_csv = os.path.join(csv_dir, "low_confidence_samples.csv")
class_accuracy_csv = os.path.join(csv_dir, "classwise_accuracy.csv")

misclassified_df.to_csv(misclassified_csv, index=False)
low_confidence_df.to_csv(low_confidence_csv, index=False)
class_accuracy_df.to_csv(class_accuracy_csv, index=False)

print(f"\n📄 검증 데이터셋에서 오답이 많은 샘플 저장됨: {misclassified_csv}")
print(f"📄 검증 데이터셋에서 확신도가 낮은 샘플 저장됨: {low_confidence_csv}")
print(f"📄 클래스별 정확도 저장됨: {class_accuracy_csv}")


✅ 사용 중인 GPU: Quadro RTX 4000

🔥 검증 데이터셋에서 문제있는 데이터 찾기 🔥

✅ 검증 데이터셋에서 자주 틀린 클래스 (상위 10개)
Class 식기·주방: 23번 오답 발생
Class 화로대·버너: 18번 오답 발생
Class 매트·침낭: 16번 오답 발생
Class 냉·난방용품: 16번 오답 발생
Class 가방·박스·웨건: 14번 오답 발생
Class 테이블·체어: 11번 오답 발생
Class 전기·전자기기: 10번 오답 발생
Class 공구·액세서리: 6번 오답 발생
Class 텐트·타프: 4번 오답 발생
Class 랜턴: 4번 오답 발생

✅ 검증 데이터셋 클래스별 정확도:
Class 가방·박스·웨건: 95.10% 정확도
Class 공구·액세서리: 95.20% 정확도
Class 냉·난방용품: 95.35% 정확도
Class 랜턴: 95.56% 정확도
Class 매트·침낭: 96.99% 정확도
Class 식기·주방: 94.38% 정확도
Class 전기·전자기기: 96.34% 정확도
Class 테이블·체어: 95.15% 정확도
Class 텐트·타프: 98.17% 정확도
Class 화로대·버너: 93.38% 정확도

✅ 검증 데이터셋에서 확신도가 낮은 샘플 (상위 10개)

📄 검증 데이터셋에서 오답이 많은 샘플 저장됨: C:/Users/user/OneDrive/Desktop/Resnet18-real/csv\misclassified_samples.csv
📄 검증 데이터셋에서 확신도가 낮은 샘플 저장됨: C:/Users/user/OneDrive/Desktop/Resnet18-real/csv\low_confidence_samples.csv
📄 클래스별 정확도 저장됨: C:/Users/user/OneDrive/Desktop/Resnet18-real/csv\classwise_accuracy.csv
